In [1]:
%matplotlib inline
local_path = '/kaggle/working/fisheries/'
data_path = '/kaggle/input/fisheries-data/data/'

batch_size=64


In [2]:
from glob import glob
import os
import numpy as np
import pandas as pd

import keras
from keras.utils.data_utils import get_file
from keras.utils.np_utils  import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,ZeroPadding2D,Lambda,Dropout,BatchNormalization
from keras.optimizers import Adam,SGD
from keras.preprocessing import image
import bcolz

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Preparing Data

In [3]:
def load_array(fname):
    return bcolz.open(fname)[:];

## KERAS VGG BN model

In [4]:
vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x=x-vgg_mean;
    return x[:,::-1]

In [5]:
def ConvBlock(layers,filters):
    for i in range(layers):
        model.add(ZeroPadding2D((1,1),data_format='channels_first'))
        model.add(Conv2D(filters, (3, 3), activation='relu',data_format='channels_first'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2),data_format="channels_first"))

In [6]:
def FCBlock():
    model.add(Dense(4096,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

In [7]:
ls /kaggle/input/

fisheries-data/    the-nature-conservancy-fisheries-monitoring/  vgg16bnconv/
fisheries-labels/  vgg16bn/                                      vgg16weights/


In [8]:
size=(224,224)
model = Sequential();
model.add(Lambda(vgg_preprocess,input_shape=(3,)+size,output_shape=(3,)+size))

ConvBlock(2,64)
ConvBlock(2,128)
ConvBlock(3,256)
ConvBlock(3,512)
ConvBlock(3,512)

model.load_weights('/kaggle/input/vgg16bnconv/vgg16_bn_conv.h5')

model.add(Flatten())
FCBlock()
FCBlock()
model.add(Dense(1000,activation='softmax'))

model.load_weights('/kaggle/input/vgg16bn/vgg16_bn.h5')

model.pop()
for layer in model.layers: layer.trainable=False;
model.add(Dense(8,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [13]:
gen=image.ImageDataGenerator();

In [11]:
trn=load_array(data_path+'trn.dat');
val=load_array(data_path+'val.dat');
test=load_array(data_path+'test.dat');


In [14]:
trn_labels = load_array(data_path+'trn_labels.dat');
val_labels = load_array(data_path+'val_labels.dat')

In [12]:
trn.shape,val.shape,test.shape

((3277, 3, 224, 224), (500, 3, 224, 224), (1000, 3, 224, 224))

In [15]:
model.fit(trn,trn_labels,epochs=3,batch_size=batch_size,validation_data=(val,val_labels))

Train on 3277 samples, validate on 500 samples
Epoch 1/3
3277/3277 [==============================] - 52s 16ms/step - loss: 2.9535 - acc: 0.4782 - val_loss: 1.9367 - val_acc: 0.5880
Epoch 2/3
3277/3277 [==============================] - 39s 12ms/step - loss: 1.6050 - acc: 0.6500 - val_loss: 1.8304 - val_acc: 0.6180
Epoch 3/3
3277/3277 [==============================] - 39s 12ms/step - loss: 1.3207 - acc: 0.7080 - val_loss: 1.7455 - val_acc: 0.6400
